In [1]:
import os
from pathlib import Path
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt

# Paths and parameters
train_dir = Path(r"C:\Users\karth\siva\Modified DATASET\train")
img_size, batch_size, epochs = (128, 128), 32, 20

In [2]:
# Function to print the directory structure
def print_directory_structure(path, level=0):
    print('    ' * level + f'[{path.name}]')
    for item in path.iterdir():
        if item.is_dir():
            print_directory_structure(item, level + 1)
        else:
            print('    ' * (level + 1) + item.name)

print("Directory structure:")
print_directory_structure(train_dir)

Directory structure:
[train]
    [Mild Impairment]
        MildImpairment (1).jpg
        MildImpairment (1005).jpg
        MildImpairment (1013).jpg
        MildImpairment (1022).jpg
        MildImpairment (1025).jpg
        MildImpairment (1037).jpg
        MildImpairment (1044).jpg
        MildImpairment (1051).jpg
        MildImpairment (1056).jpg
        MildImpairment (1062).jpg
        MildImpairment (1069).jpg
        MildImpairment (1072).jpg
        MildImpairment (1085).jpg
        MildImpairment (1088).jpg
        MildImpairment (1103).jpg
        MildImpairment (1104).jpg
        MildImpairment (1108).jpg
        MildImpairment (1120).jpg
        MildImpairment (1122).jpg
        MildImpairment (1125).jpg
        MildImpairment (1137).jpg
        MildImpairment (1143).jpg
        MildImpairment (1150).jpg
        MildImpairment (1151).jpg
        MildImpairment (1158).jpg
        MildImpairment (1180).jpg
        MildImpairment (1185).jpg
        MildImpairment (1203).jpg


In [3]:
# Data generators with augmentation
datagen = ImageDataGenerator(
    rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True, validation_split=0.2
)
train_gen = datagen.flow_from_directory(
    train_dir, target_size=img_size, batch_size=batch_size, class_mode='categorical', subset='training'
)
val_gen = datagen.flow_from_directory(
    train_dir, target_size=img_size, batch_size=batch_size, class_mode='categorical', subset='validation'
)

print(f"Training samples: {train_gen.samples}, Validation samples: {val_gen.samples}")
print(f"Classes: {len(train_gen.class_indices)}, Class indices: {train_gen.class_indices}")


Found 8090 images belonging to 4 classes.
Found 2021 images belonging to 4 classes.
Training samples: 8090, Validation samples: 2021
Classes: 4, Class indices: {'Mild Impairment': 0, 'Moderate Impairment': 1, 'No Impairment': 2, 'Very Mild Impairment': 3}


In [4]:
import tensorflow as tf
from tensorflow.keras import layers, models, regularizers

def create_improved_cnn(img_size, num_classes):
    model = models.Sequential([
        # Initial convolution block
        layers.Conv2D(64, (3, 3), activation='relu', padding='same', input_shape=(img_size[0], img_size[1], 3)),
        layers.BatchNormalization(),
        layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.MaxPooling2D(2, 2),
        layers.Dropout(0.3),

        # Second convolution block
        layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.MaxPooling2D(2, 2),
        layers.Dropout(0.4),

        # Third convolution block
        layers.Conv2D(256, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.Conv2D(256, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.Conv2D(256, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.MaxPooling2D(2, 2),
        layers.Dropout(0.4),

        # Fourth convolution block
        layers.Conv2D(512, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.Conv2D(512, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.Conv2D(512, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.MaxPooling2D(2, 2),
        layers.Dropout(0.4),

        # Fifth convolution block
        layers.Conv2D(512, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.Conv2D(512, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.Conv2D(512, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.MaxPooling2D(2, 2),
        layers.Dropout(0.4),

        # Flatten and Dense layers
        layers.Flatten(),
        layers.Dense(4096, activation='relu', kernel_regularizer=regularizers.l2(0.001)),
        layers.BatchNormalization(),
        layers.Dropout(0.5),
        layers.Dense(4096, activation='relu', kernel_regularizer=regularizers.l2(0.001)),
        layers.BatchNormalization(),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation='softmax')
    ])

    return model

# Create and compile the model
img_size = (128, 128)  # Adjust this to match your input image size
num_classes = len(train_gen.class_indices)
model = create_improved_cnn(img_size, num_classes)

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Print model summary
model.summary()

C:\Users\karth\siva\virt\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 128, 128, 64)        │           1,792 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization                  │ (None, 128, 128, 64)        │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 128, 128, 64)        │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_1                │ (None, 128, 128, 64)        │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 64, 64, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 64, 64, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 64, 64, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_2                │ (None, 64, 64, 128)         │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_3 (Conv2D)                    │ (None, 64, 64, 128)         │         147,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_3                │ (None, 64, 64, 128)         │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 32, 32, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 32, 32, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_4 (Conv2D)                    │ (None, 32, 32, 256)         │         295,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_4                │ (None, 32, 32, 256)         │           1,024 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_5 (Conv2D)                    │ (None, 32, 32, 256)         │         590,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_5                │ (None, 32, 32, 256)         │           1,024 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_6 (Conv2D)                    │ (None, 32, 32, 256)         │         590,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_6                │ (None, 32, 32, 256)         │           1,024 │
│ (BatchNormalization)                 │                             │              

 Total params: 65,120,580 (248.42 MB)

 Trainable params: 65,095,748 (248.32 MB)

 Non-trainable params: 24,832 (97.00 KB)

In [5]:
# Train the model
history = model.fit(train_gen, epochs=epochs, validation_data=val_gen)
print("Model training completed.")


C:\Users\karth\siva\virt\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20
111/253 ━━━━━━━━━━━━━━━━━━━━ 11:41 5s/step - accuracy: 0.2969 - loss: 11.9237

KeyboardInterrupt: 

In [7]:
import os
from pathlib import Path
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt

# GPU Configuration
def configure_gpu():
    # Check if GPU is available
    gpus = tf.config.list_physical_devices('GPU')
    if gpus:
        try:
            # Enable memory growth to avoid allocating all GPU memory at once
            for gpu in gpus:
                tf.config.experimental.set_memory_growth(gpu, True)
            
            # Set mixed precision policy for faster training
            tf.keras.mixed_precision.set_global_policy('mixed_float16')
            
            print(f"Found {len(gpus)} GPU(s). GPU configuration successful.")
        except RuntimeError as e:
            print(f"GPU configuration error: {e}")
    else:
        print("No GPU found. Running on CPU.")

# Configure GPU before any other operations
configure_gpu()

# Paths and parameters
train_dir = Path(r"C:\Users\karth\siva\Modified DATASET\train")
img_size, batch_size, epochs = (128, 128), 32, 20

# Function to print the directory structure
def print_directory_structure(path, level=0):
    print('    ' * level + f'[{path.name}]')
    for item in path.iterdir():
        if item.is_dir():
            print_directory_structure(item, level + 1)
        else:
            print('    ' * (level + 1) + item.name)

print("Directory structure:")
print_directory_structure(train_dir)

# Optimize data loading with tf.data
def create_dataset(generator, is_training=True):
    return tf.data.Dataset.from_generator(
        lambda: generator,
        output_types=(tf.float32, tf.float32),
        output_shapes=([batch_size, img_size[0], img_size[1], 3], [batch_size, None])
    ).prefetch(tf.data.AUTOTUNE)

# Data generators with augmentation
datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2
)

train_gen = datagen.flow_from_directory(
    train_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

val_gen = datagen.flow_from_directory(
    train_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)

# Create optimized datasets
train_dataset = create_dataset(train_gen)
val_dataset = create_dataset(val_gen, is_training=False)

print(f"Training samples: {train_gen.samples}, Validation samples: {val_gen.samples}")
print(f"Classes: {len(train_gen.class_indices)}, Class indices: {train_gen.class_indices}")

def create_improved_cnn(img_size, num_classes):
    # Use float16 for faster computation while keeping certain layers in float32
    model = models.Sequential([
        # Initial convolution block
        layers.Conv2D(64, (3, 3), activation='relu', padding='same', 
                     input_shape=(img_size[0], img_size[1], 3), dtype='float16'),
        layers.BatchNormalization(dtype='float16'),
        layers.Conv2D(64, (3, 3), activation='relu', padding='same', dtype='float16'),
        layers.BatchNormalization(dtype='float16'),
        layers.MaxPooling2D(2, 2),
        layers.Dropout(0.3),
        
        # Second convolution block
        layers.Conv2D(128, (3, 3), activation='relu', padding='same', dtype='float16'),
        layers.BatchNormalization(dtype='float16'),
        layers.Conv2D(128, (3, 3), activation='relu', padding='same', dtype='float16'),
        layers.BatchNormalization(dtype='float16'),
        layers.MaxPooling2D(2, 2),
        layers.Dropout(0.4),
        
        # Third convolution block
        layers.Conv2D(256, (3, 3), activation='relu', padding='same', dtype='float16'),
        layers.BatchNormalization(dtype='float16'),
        layers.Conv2D(256, (3, 3), activation='relu', padding='same', dtype='float16'),
        layers.BatchNormalization(dtype='float16'),
        layers.Conv2D(256, (3, 3), activation='relu', padding='same', dtype='float16'),
        layers.BatchNormalization(dtype='float16'),
        layers.MaxPooling2D(2, 2),
        layers.Dropout(0.4),
        
        # Fourth and Fifth blocks similar pattern...
                # Fourth convolution block
        layers.Conv2D(512, (3, 3), activation='relu', padding='same', dtype='float16'),
        layers.BatchNormalization(dtype='float16'),
        layers.Conv2D(512, (3, 3), activation='relu', padding='same', dtype='float16'),
        layers.BatchNormalization(dtype='float16'),
        layers.Conv2D(512, (3, 3), activation='relu', padding='same', dtype='float16'),
        layers.BatchNormalization(dtype='float16'),
        layers.MaxPooling2D(2, 2),
        layers.Dropout(0.4),
        
        # Converting back to float32 for the dense layers
        layers.Flatten(),
        layers.Dense(4096, activation='relu', 
                    kernel_regularizer=regularizers.l2(0.001), dtype='float32'),
        layers.BatchNormalization(),
        layers.Dropout(0.5),
        layers.Dense(4096, activation='relu', 
                    kernel_regularizer=regularizers.l2(0.001), dtype='float32'),
        layers.BatchNormalization(),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation='softmax', dtype='float32')
    ])
    return model

# Create and compile the model with mixed precision optimizer
img_size = (128, 128)
num_classes = len(train_gen.class_indices)
model = create_improved_cnn(img_size, num_classes)

# Use mixed precision optimizer wrapper
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)
optimizer = tf.keras.mixed_precision.LossScaleOptimizer(optimizer)

model.compile(
    optimizer=optimizer,
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Print model summary
model.summary()

# ... (previous imports and GPU configuration remain the same)

# Custom training callback to monitor GPU memory
class GPUMonitor(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        gpus = tf.config.list_physical_devices('GPU')
        for gpu in gpus:
            gpu_mem = tf.config.experimental.get_memory_info(gpu)['current']
            print(f"\nGPU Memory Usage: {gpu_mem / 1024**2:.2f} MB")

# Train the model with GPU monitoring
callbacks = [
    GPUMonitor(),
    tf.keras.callbacks.ModelCheckpoint(
        'best_model.keras',  # Changed from .h5 to .keras
        save_best_only=True,
        monitor='val_accuracy'
    ),
    tf.keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.2,
        patience=3
    )
]

history = model.fit(
    train_dataset,
    epochs=epochs,
    validation_data=val_dataset,
    callbacks=callbacks
)

print("Model training completed.")

# Save the final model
model.save('final_model.keras')  # Changed from .h5 to .keras

No GPU found. Running on CPU.
Directory structure:
[train]
    [Mild Impairment]
        MildImpairment (1).jpg
        MildImpairment (1005).jpg
        MildImpairment (1013).jpg
        MildImpairment (1022).jpg
        MildImpairment (1025).jpg
        MildImpairment (1037).jpg
        MildImpairment (1044).jpg
        MildImpairment (1051).jpg
        MildImpairment (1056).jpg
        MildImpairment (1062).jpg
        MildImpairment (1069).jpg
        MildImpairment (1072).jpg
        MildImpairment (1085).jpg
        MildImpairment (1088).jpg
        MildImpairment (1103).jpg
        MildImpairment (1104).jpg
        MildImpairment (1108).jpg
        MildImpairment (1120).jpg
        MildImpairment (1122).jpg
        MildImpairment (1125).jpg
        MildImpairment (1137).jpg
        MildImpairment (1143).jpg
        MildImpairment (1150).jpg
        MildImpairment (1151).jpg
        MildImpairment (1158).jpg
        MildImpairment (1180).jpg
        MildImpairment (1185).jpg
    

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_23 (Conv2D)                   │ (None, 128, 128, 64)        │           1,792 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_27               │ (None, 128, 128, 64)        │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_24 (Conv2D)                   │ (None, 128, 128, 64)        │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_28               │ (None, 128, 128, 64)        │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_9 (MaxPooling2D)       │ (None, 64, 64, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_13 (Dropout)                 │ (None, 64, 64, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_25 (Conv2D)                   │ (None, 64, 64, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_29               │ (None, 64, 64, 128)         │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_26 (Conv2D)                   │ (None, 64, 64, 128)         │         147,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_30               │ (None, 64, 64, 128)         │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_10 (MaxPooling2D)      │ (None, 32, 32, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_14 (Dropout)                 │ (None, 32, 32, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_27 (Conv2D)                   │ (None, 32, 32, 256)         │         295,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_31               │ (None, 32, 32, 256)         │           1,024 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_28 (Conv2D)                   │ (None, 32, 32, 256)         │         590,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_32               │ (None, 32, 32, 256)         │           1,024 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_29 (Conv2D)                   │ (None, 32, 32, 256)         │         590,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_33               │ (None, 32, 32, 256)         │           1,024 │
│ (BatchNormalization)                 │                             │              

 Total params: 158,698,308 (590.80 MB)

 Trainable params: 158,676,548 (590.73 MB)

 Non-trainable params: 21,760 (74.50 KB)

Epoch 1/20
      2/Unknown 327s 164s/step - accuracy: 0.3359 - loss: 13.9797

KeyboardInterrupt: 

In [ ]:
import os
import tensorflow as tf
from pathlib import Path
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import numpy as np

# GPU Configuration
def configure_gpu():
    try:
        # Print TensorFlow version and CUDA availability
        print(f"TensorFlow version: {tf.__version__}")
        print(f"CUDA available: {tf.test.is_built_with_cuda()}")
        
        # List all available GPUs
        gpus = tf.config.list_physical_devices('GPU')
        if gpus:
            for gpu in gpus:
                print(f"Found GPU: {gpu}")
                
            # Try to set memory growth
            try:
                for gpu in gpus:
                    tf.config.experimental.set_memory_growth(gpu, True)
                print("Memory growth enabled")
            except RuntimeError as e:
                print(f"Memory growth setting failed: {e}")
            
            # Set visible devices
            tf.config.set_visible_devices(gpus[0], 'GPU')
            
            # Set memory limit (adjust based on your GPU memory, e.g., 4GB = 4096)
            try:
                tf.config.set_logical_device_configuration(
                    gpus[0],
                    [tf.config.LogicalDeviceConfiguration(memory_limit=4096)])
            except RuntimeError as e:
                print(f"Memory limit setting failed: {e}")
                
            # Verify GPU is being used
            print("GPU Device:", tf.test.gpu_device_name())
            
            # Enable mixed precision
            policy = tf.keras.mixed_precision.Policy('mixed_float16')
            tf.keras.mixed_precision.set_global_policy(policy)
            print("Mixed precision enabled")
            
        else:
            print("No GPU devices found!")
            
    except Exception as e:
        print(f"GPU configuration error: {e}")
        print("Falling back to CPU")

# Configure GPU before any other operations
configure_gpu()

# Test GPU with a simple operation
def verify_gpu_operation():
    try:
        with tf.device('/GPU:0'):
            a = tf.constant([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])
            b = tf.constant([[1.0, 2.0], [3.0, 4.0], [5.0, 6.0]])
            c = tf.matmul(a, b)
            print("\nGPU test successful. Matrix multiplication result:")
            print(c)
    except Exception as e:
        print(f"GPU test failed: {e}")

verify_gpu_operation()

# Paths and parameters - adjust batch size based on GPU memory
train_dir = Path(r"C:\Users\karth\siva\Modified DATASET\train")
img_size = (128, 128)
batch_size = 16  # Reduced batch size for RTX 2050
epochs = 20

# Optimize data loading for GPU
AUTOTUNE = tf.data.AUTOTUNE

def optimize_data_loading(dataset):
    return dataset.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)

# Data generators with augmentation
datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2
)

train_gen = datagen.flow_from_directory(
    train_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

val_gen = datagen.flow_from_directory(
    train_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)

# Create tf.data.Dataset for better GPU utilization
def create_dataset(generator):
    return tf.data.Dataset.from_generator(
        lambda: generator,
        output_types=(tf.float32, tf.float32),
        output_shapes=([batch_size, img_size[0], img_size[1], 3], 
                      [batch_size, len(generator.class_indices)])
    ).apply(optimize_data_loading)

train_dataset = create_dataset(train_gen)
val_dataset = create_dataset(val_gen)

# Model creation with GPU optimization
def create_improved_cnn(img_size, num_classes):
    with tf.device('/GPU:0'):
        model = tf.keras.Sequential([
           # Initial convolution block
        layers.Conv2D(64, (3, 3), activation='relu', padding='same', 
                     input_shape=(img_size[0], img_size[1], 3), dtype='float16'),
        layers.BatchNormalization(dtype='float16'),
        layers.Conv2D(64, (3, 3), activation='relu', padding='same', dtype='float16'),
        layers.BatchNormalization(dtype='float16'),
        layers.MaxPooling2D(2, 2),
        layers.Dropout(0.3),
        
        # Second convolution block
        layers.Conv2D(128, (3, 3), activation='relu', padding='same', dtype='float16'),
        layers.BatchNormalization(dtype='float16'),
        layers.Conv2D(128, (3, 3), activation='relu', padding='same', dtype='float16'),
        layers.BatchNormalization(dtype='float16'),
        layers.MaxPooling2D(2, 2),
        layers.Dropout(0.4),
        
        # Third convolution block
        layers.Conv2D(256, (3, 3), activation='relu', padding='same', dtype='float16'),
        layers.BatchNormalization(dtype='float16'),
        layers.Conv2D(256, (3, 3), activation='relu', padding='same', dtype='float16'),
        layers.BatchNormalization(dtype='float16'),
        layers.Conv2D(256, (3, 3), activation='relu', padding='same', dtype='float16'),
        layers.BatchNormalization(dtype='float16'),
        layers.MaxPooling2D(2, 2),
        layers.Dropout(0.4),
        
        # Fourth and Fifth blocks similar pattern...
                # Fourth convolution block
        layers.Conv2D(512, (3, 3), activation='relu', padding='same', dtype='float16'),
        layers.BatchNormalization(dtype='float16'),
        layers.Conv2D(512, (3, 3), activation='relu', padding='same', dtype='float16'),
        layers.BatchNormalization(dtype='float16'),
        layers.Conv2D(512, (3, 3), activation='relu', padding='same', dtype='float16'),
        layers.BatchNormalization(dtype='float16'),
        layers.MaxPooling2D(2, 2),
        layers.Dropout(0.4),
        
        # Converting back to float32 for the dense layers
        layers.Flatten(),
        layers.Dense(4096, activation='relu', 
                    kernel_regularizer=regularizers.l2(0.001), dtype='float32'),
        layers.BatchNormalization(),
        layers.Dropout(0.5),
        layers.Dense(4096, activation='relu', 
                    kernel_regularizer=regularizers.l2(0.001), dtype='float32'),
        layers.BatchNormalization(),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation='softmax', dtype='float32')
    
        ])
    return model

# Create and compile model with GPU optimization
strategy = tf.distribute.OneDeviceStrategy("/GPU:0")
with strategy.scope():
    model = create_improved_cnn(img_size, len(train_gen.class_indices))
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)
    
    # Enable mixed precision
    optimizer = tf.keras.mixed_precision.LossScaleOptimizer(optimizer)
    
    model.compile(
        optimizer=optimizer,
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )

# Custom callback for GPU monitoring
class GPUMonitor(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        try:
            gpus = tf.config.list_physical_devices('GPU')
            for gpu in gpus:
                # Print GPU memory info
                memory_info = tf.config.experimental.get_memory_info('GPU:0')
                print(f"\nGPU Memory Usage: {memory_info['current'] / 1024**2:.2f} MB")
        except Exception as e:
            print(f"GPU monitoring error: {e}")

# Training with GPU monitoring
callbacks = [
    GPUMonitor(),
    tf.keras.callbacks.ModelCheckpoint(
        'best_model.keras',
        save_best_only=True,
        monitor='val_accuracy'
    ),
    tf.keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.2,
        patience=3
    ),
    tf.keras.callbacks.EarlyStopping(
        monitor='val_loss',
        patience=5,
        restore_best_weights=True
    )
]

# Train the model
try:
    with tf.device('/GPU:0'):
        history = model.fit(
            train_dataset,
            epochs=epochs,
            validation_data=val_dataset,
            callbacks=callbacks,
            use_multiprocessing=True,
            workers=4
        )
    print("Training completed successfully")
except Exception as e:
    print(f"Training error: {e}")

# Save the final model
model.save('final_model.keras')